#### Conceptos de este cuaderno :

## Con que trabaja spark?

##### DataSet(Datos para explotar, normalmente en formato CSV, JSON o similares). Los 'DataFrames'(No distribuido, mutable y no tolerante a fallos) y los 'RDD(Resilient Distributed Dataset)'(Distribuidos, inmutable y tolerante a fallos).

#### En estos primeras celdas trabajaremos con Apache Spark en modo 'Standalone', es decir, que tendremos todo corriendo en una sola máquina.

In [11]:
# Que version de Pyspark estamos usando? 

import pyspark
print(pyspark.__version__)

3.4.1


In [18]:
# Que version de Python estamos usando? 
import sys
print(sys.version)

3.9.13 (main, Aug 25 2022, 18:29:29) 
[Clang 12.0.0 ]


In [8]:
# crear un dataframe de cliente declarando el esquema y pasando valores
import pyspark.sql.functions as F
from pyspark.sql.types import *

from pyspark.sql import SparkSession

In [9]:
# Crear una sesión de Spark
spark = SparkSession.builder.appName("EcommerceCosmeticShop").getOrCreate()

# terminal 
#Arrancamos el master en modo standalone(Arranca WebUI en 127.0.0.1:8080)
$SPARK_HOME/sbin/start-master.sh

 Arrancamos el primer esclavo
$SPARK_HOME/sbin/start-slave.sh spark://MacBook-Pro-de-Lucas-880.local:7077

In [ ]:
# Leemos datos del DataSet y los escribimos en un DataFrame
df = spark.read.options(header='True', inferSchema='True').csv(['2019-Dec.csv', '2019-Nov.csv', '2019-Oct.csv', '2020-Feb.csv', '2020-Jan.csv'])
# Escribimos el DataFrame en disco en la carpeta donde se encuentra este Jupyter Notebook
df.write.mode('overwrite').csv("ecommerce-cosmetic-shop")


##### Detener la sesión de Spark cuando hayas terminado
spark.stop()


Comprobar que el proceso ha generado ficheros CSVs. El motivo es que Apache Spark particiona los datos para procesar en paralelo y, por lo tanto, crea un fichero por partición(También ha creado un fichero '_SUCCESS' para indicar el momento en que había acabado de exportar).

In [ ]:
# Contar número de registros de un DataFrame
df.count()

In [ ]:
# Escribir el Schema de los datos del Data Frame
df.printSchema()

In [ ]:
# Mostrar o contar los diferentes valores de una columna
df.select('event_type').distinct().show()

In [ ]:
# Mostrar o contar los diferentes valores de una columna
df.select('product_id').distinct().count()

In [ ]:
df.select('brand').distinct().show()

In [ ]:
# Obtener el primer 'product_id' del registro con 'event_type=cart'
df.select(['product_id']).filter("event_type='cart'").first()

In [ ]:
# Obtener los productos que se han comprado conjuntamente con el anterior registro
sesions=df.select(['user_session']).filter("event_type='cart' AND product_id=5844305").distinct().first()

In [ ]:
products=df.select(['product_id']).filter("event_type='cart' AND product_id<>5844305").filter( df["user_session"].isin(sesions["user_session"]))
products.select('product_id').show()

### SQL API - opcion b

In [ ]:
df.createOrReplaceTempView("data")
spark.sql("select * from data limit 5").show()

In [ ]:
def myFunc(s):
  if s["brand"]=="riche" and s["event_type"]=="cart":
    return [ ( s["product_id"], 1) ]
  return []

In [ ]:
lines=df.rdd.flatMap(myFunc).reduceByKey(lambda a, b: a + b)

In [ ]:
for element in lines.collect(): 
  print(element)